# Importing Dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import cv2 
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Loading Dataset

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
direc=tf.keras.utils.get_file('flower_photos',origin=dataset_url,cache_dir='.', untar=True)

228813984/228813984 [==============================] - 2s 0us/step


# Global Path

In [3]:
direc

'./datasets/flower_photos'

In [4]:
direc=pathlib.Path(direc)
direc

PosixPath('datasets/flower_photos')

In [5]:
list(direc.glob('*/*.jpg'))

[PosixPath('datasets/flower_photos/daisy/14866200659_6462c723cb_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/134409839_71069a95d1_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/1306119996_ab8ae14d72_n.jpg'),
 PosixPath('datasets/flower_photos/daisy/13583238844_573df2de8e_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/14569895116_32f0dcb0f9.jpg'),
 PosixPath('datasets/flower_photos/daisy/18354545086_693ea7bc2a.jpg'),
 PosixPath('datasets/flower_photos/daisy/2901376034_cdb4bac26b_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/7066602021_2647457985_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/3098641292_76c908ba1f_n.jpg'),
 PosixPath('datasets/flower_photos/daisy/2365428551_39f83f10bf_n.jpg'),
 PosixPath('datasets/flower_photos/daisy/3661613900_b15ca1d35d_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/16291797949_a1b1b7c2bd_n.jpg'),
 PosixPath('datasets/flower_photos/daisy/18582579815_4c6637e9ff_m.jpg'),
 PosixPath('datasets/flower_photos/daisy/5796562389_ae43c83317_

In [6]:
count=len(list(direc.glob('*/*.jpg')))

# Storing all flowers

In [7]:
flowers={
    'rose':list(direc.glob('roses/*')),
    'tulip':list(direc.glob('tulips/*')),
    'dandelion':list(direc.glob('dandelion/*')),
    'daisy':list(direc.glob('daisy/*')),
    'sunflower':list(direc.glob('sunflowers/*')),
}

In [8]:
lables={
    'rose':0,
    'daisy':1,
    'dandelion':2,
    'sunflower':3,
    'tulip':4,
    
}

In [9]:
image=cv2.imread(str(flowers['rose'][0]))

In [10]:
image.shape

(240, 320, 3)

Reading and resizing images from a dataset of flowers, storing the resized images in list x and their corresponding labels in list y. The labels are determined by mapping flower names to labels using the labels dictionary.

In [11]:
x,y=[],[]
for flower_name,images in flowers.items():
    for img in images:
        img=cv2.imread(str(img))
        resized=cv2.resize(img,(100,100))
        x.append(resized)
        y.append(lables[flower_name])
    

In [12]:
x=np.array(x)
y=np.array(y)

In [13]:
x.shape,y.shape

((3670, 100, 100, 3), (3670,))

# Train Test Split

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0)

In [15]:
x_train.shape,x_test.shape

((2752, 100, 100, 3), (918, 100, 100, 3))

# Scaling Images

In [16]:
x_train_scaled=x_train/255
x_test_scaled=x_test/255

# Model Building

In [17]:
classes=5
model=Sequential([
    layers.Conv2D(16,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(classes)
])

model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

# Model Training

In [18]:
model.fit(x_train_scaled,y_train,epochs=20)

Epoch 1/20
86/86 [==============================] - 16s 169ms/step - loss: 1.2361 - accuracy: 0.4633
Epoch 2/20
86/86 [==============================] - 15s 169ms/step - loss: 0.9926 - accuracy: 0.5999
Epoch 3/20
86/86 [==============================] - 15s 169ms/step - loss: 0.8578 - accuracy: 0.6690
Epoch 4/20
86/86 [==============================] - 15s 169ms/step - loss: 0.7363 - accuracy: 0.7206
Epoch 5/20
86/86 [==============================] - 15s 171ms/step - loss: 0.6224 - accuracy: 0.7656
Epoch 6/20
86/86 [==============================] - 14s 163ms/step - loss: 0.4866 - accuracy: 0.8245
Epoch 7/20
86/86 [==============================] - 15s 172ms/step - loss: 0.3634 - accuracy: 0.8725
Epoch 8/20
86/86 [==============================] - 14s 167ms/step - loss: 0.2458 - accuracy: 0.9186
Epoch 9/20
86/86 [==============================] - 15s 172ms/step - loss: 0.1651 - accuracy: 0.9484
Epoch 10/20
86/86 [==============================] - 14s 167ms/step - loss: 0.1091 - accura

# Test Data Evaluation

In [19]:
model.evaluate(x_test_scaled,y_test)

29/29 [==============================] - 2s 44ms/step - loss: 1.7165 - accuracy: 0.6612


[1.716465950012207, 0.6612200140953064]

In [20]:
predictions=model.predict(x_test_scaled)

29/29 [==============================] - 1s 43ms/step


In [21]:
predictions

array([[  6.0495896 ,  -5.2500305 ,  -4.832042  ,  -7.5773144 ,
         17.513409  ],
       [  6.3124747 ,   5.268334  ,   0.48024994, -11.737869  ,
          2.0117652 ],
       [ -2.9519315 ,  -3.1161592 ,   5.7984667 ,  -0.9318184 ,
          0.56333476],
       ...,
       [-11.171617  ,  13.206517  ,   4.081371  ,  -4.1494527 ,
         -1.965937  ],
       [ 14.795823  ,  -8.069537  ,  -0.6375924 ,  -1.5225643 ,
          4.931836  ],
       [ -2.7246072 ,   8.504483  ,  -0.9589247 ,   6.5074635 ,
         -2.1584308 ]], dtype=float32)

# Data Augmentation

In [22]:
data_augmentation=keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(100,100,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
    
])

In [23]:
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(x_train_scaled, y_train, epochs=20)  

Epoch 1/20
86/86 [==============================] - 19s 204ms/step - loss: 1.3145 - accuracy: 0.4259
Epoch 2/20
86/86 [==============================] - 18s 208ms/step - loss: 1.0628 - accuracy: 0.5749
Epoch 3/20
86/86 [==============================] - 18s 209ms/step - loss: 0.9507 - accuracy: 0.6294
Epoch 4/20
86/86 [==============================] - 18s 204ms/step - loss: 0.8800 - accuracy: 0.6610
Epoch 5/20
86/86 [==============================] - 18s 208ms/step - loss: 0.8544 - accuracy: 0.6839
Epoch 6/20
86/86 [==============================] - 18s 208ms/step - loss: 0.7951 - accuracy: 0.6879
Epoch 7/20
86/86 [==============================] - 18s 210ms/step - loss: 0.7604 - accuracy: 0.7089
Epoch 8/20
86/86 [==============================] - 17s 201ms/step - loss: 0.7247 - accuracy: 0.7238
Epoch 9/20
86/86 [==============================] - 18s 207ms/step - loss: 0.7118 - accuracy: 0.7275
Epoch 10/20
86/86 [==============================] - 17s 200ms/step - loss: 0.6894 - accura

In [24]:
model.evaluate(x_test_scaled,y_test)

29/29 [==============================] - 1s 44ms/step - loss: 0.8493 - accuracy: 0.7113


[0.8493126034736633, 0.7113289833068848]